Номер группы, ФИО: М8О-202М-21, Базыльникова Екатерина Родионовна

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as p

In [18]:
wine = datasets.load_wine()
XX = wine.data
yy = wine.target
X_train, X_test, y_train, y_test = train_test_split(XX, yy, test_size=0.3, random_state=5)

In [19]:
classifiers = [Perceptron(max_iter=1000, tol=1e-3), LogisticRegression(multi_class='auto', solver='liblinear'),
               KNeighborsClassifier(n_neighbors=5), DecisionTreeClassifier(max_depth=3)]

In [20]:
def acc_for_clrs(X_train, y_train, X_test, y_test):
  for classifier in classifiers:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(classifier.__class__.__name__, accuracy_score(y_test, y_pred))

In [22]:
standard_scaler = StandardScaler()
s_scaled = p.DataFrame(standard_scaler.fit_transform(X_train))
s_scaled_test = p.DataFrame(standard_scaler.transform(X_test))

In [24]:
minmax_scaler = MinMaxScaler()
mm_scaled = p.DataFrame(minmax_scaler.fit_transform(X_train))
mm_scaled_test = p.DataFrame(minmax_scaler.transform(X_test))

Точность работы классификаторов:
1. без масштабирования
2. с масштабированием standard_scaler
3. с масштабированием minmaxscaler

In [25]:
print("=== No scaler ===")
acc_for_clrs(X_train, y_train, X_test, y_test)
print("=== Standard scaler ===")
acc_for_clrs(s_scaled, y_train, s_scaled_test, y_test)
print("=== Minmax scaler ===")
acc_for_clrs(mm_scaled, y_train, mm_scaled_test, y_test)

=== No scaler ===
Perceptron 0.5925925925925926
LogisticRegression 0.9259259259259259
KNeighborsClassifier 0.7037037037037037
DecisionTreeClassifier 0.9259259259259259
=== Standard scaler ===
Perceptron 1.0
LogisticRegression 1.0
KNeighborsClassifier 0.9629629629629629
DecisionTreeClassifier 0.9259259259259259
=== Minmax scaler ===
Perceptron 0.9259259259259259
LogisticRegression 0.9444444444444444
KNeighborsClassifier 0.9259259259259259
DecisionTreeClassifier 0.9259259259259259


Ответить на вопросы:
1. В каком случае (StandatdScaler/MinMaxScaler/без масштабирования) алгоритмы показали лучший результат?

Ответ: <font color='green'>
*   Персептрон показал наибольшую точность (1) на масштабированных данных (стандартное масштабирование), после идет минмакс.
*   Для логистической регрессии все три случая практически одинаковы, стандартное масштабирование дало лучший результат (0.96), отсутствие масштабирования -- худший (0.92).
* Метод ближайших соседей лучше всего показал себя на стандартно масштабированных данных, чуть хуже минмакс, хуже всех -- без масштабирования.
* Дерево решений показало одинаковый результат во всех случаях </font>


2. Как повлияло масштабирование признаков на результаты работы DecisionTreeClassifier?

Ответ: <font color='green'>Никак (возможно потому что дерево создает фиксированное количество узлов?)</font>

Для классификатора `LogisticRegression` сделать конвейер с лучшим алгоритмом масштабирования, полученным в предыдущем задании. Обучить конвейер на тренировочных данных, сделать предсказания с помощью конвейера, оценить точность на тестовых данных.
Из конвейера получить натренированный классификатор (`pipline.steps[0]` вернет `tuple` с масштабирующим классом, `pipline.steps[1]` вернет `tuple` с классификатором, откуда можно получить классификатор)

In [31]:
from sklearn.pipeline import Pipeline
steps = [('scaler',standard_scaler), ('classifier', classifiers[1])]
pipe = Pipeline(steps)
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(accuracy_score(y_test, y_pred))

1.0


Отобразить коэффициенты весов `w` разделяющих классы гиперплоскостей (3 массива весов, которые отделяют каждый класс от двух других)

In [47]:
print(pipe.steps[1][1].coef_)
weights = np.array(pipe.steps[1][1].coef_)

[[ 1.19492626  0.13597831  0.47455935 -1.27276261 -0.00591202  0.31594293
   0.92846648 -0.26177061 -0.0957287   0.13128945  0.06106702  0.64448827
   1.72510196]
 [-1.39030608 -0.50592331 -0.80222654  0.64718261 -0.04381452 -0.10982866
   0.49686606  0.29667133  0.21979153 -1.89590739  0.84135887  0.08353773
  -1.65794101]
 [ 0.33611661  0.35644895  0.32539937  0.32404121  0.07326638 -0.01396881
  -1.31261802 -0.03135595 -0.33526387  1.65677635 -0.83741372 -0.79692207
  -0.13077839]]


In [48]:
names = np.array(['Алкоголь','Яблочная кислота','Зола','Щелочность золы','Магнезия',
                  'Общее содержание фенолов', 'Флаваноиды', 'Нефлаваноидные фенолы', 
                  'Проантоцианины', 'Интенсивность цвета', 'Оттенок', 
                  'OD280 / OD315 разбавленных (разведенных) вин', 'Пролин'])

По коэффициентам весов `w` классификатора  определить, какие из признаков наиболее значимы (имеют абсолютное значение больше единицы). <b>Важно:</b> для отбора признаков использовать возможности библиотеки NumPy (см. lection_2_numpy_start)

In [68]:
for i in range(0,3):
  tfvector = np.abs(weights[i])>1
  print(names[tfvector == True])

['Алкоголь' 'Щелочность золы' 'Пролин']
['Алкоголь' 'Интенсивность цвета' 'Пролин']
['Флаваноиды' 'Интенсивность цвета']
